`ПАРСИНГ САЙТА 'rawg.io' ЧЕРЕЗ API С РЕЙТИНГОМ ИГР И ИХ ЖАНРАМИ`

In [49]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver as wd
import pandas as pd

In [41]:
API_key = 'f74b1887af7c4db5b574c79563e32354'
url = 'https://api.rawg.io/api/games'

parametrs = {
    'key': API_key,
    'page_size': 40,
}

page = requests.get(url, params=parametrs)
res = page.json()

print(res.keys()) # Смотрим на ключи в JSON формате
print(res['results']) # В ключе results оказывается важная для нас информация об играх, будет работать с этим ключом

In [47]:
all_games = []
page = 350 # Спарсим только первые 15 000 игр, которые имеют наибольшее число скачиваний

for page in range(1, page + 1):
    parametrs['page'] = page
    response = requests.get(url, params=parametrs)
    data = response.json()
    for game in data['results']:
        all_games.append({
            'name': game.get('name'),
            'released': game.get('released'),
            'rating': game.get('rating'),
            'esrb_rating': game.get('esrb_rating'),
            'genres': ', '.join(x['name'] for x in game.get('genres')),
            'platforms': ', '.join(y['platform']['name'] for y in game.get('platforms'))
        })

print(len(all_games))

In [53]:
df = pd.DataFrame(all_games)
df['esrb_rating'] = df['esrb_rating'].apply(lambda x: x.get('name') if x else pd.NA)
df.to_csv('games.csv', index=False)
df.head(10)

----------------

`ПАРСИНГ САЙТА 'gamalytic' ЧЕРЕЗ WEB SCRAPING (С ИСПОЛЬЗОВАНИЕМ Selenium) С ДАННЫМИ О ПРИБЫЛЬНОСТИ ИГР`

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By # позволяет искать элементы на странице
from selenium.webdriver.chrome.options import Options # позволяет задавать параметры запуска Chrome
from selenium.webdriver.chrome.service import Service # вспомогательный класс для контроля выполнения команд selenium внутри браузере
from selenium.webdriver.support.ui import WebDriverWait # ждёт загрузку сайта
from selenium.webdriver.support import expected_conditions as ec # позволяет дождаться появления нужных элементов на странице
from webdriver_manager.chrome import ChromeDriverManager # скачивает и обновляет ChromeDriverManager, через который selenium управляет браузером

from pathlib import Path
import time
import csv

URL = 'https://gamalytic.com/game-list?columns=headerUrl%2Cname%2CfirstReleaseDate%2CcopiesSold%2Cprice%2Crevenue%2CavgPlaytime%2CreviewScore%2Creviews%2Cfollowers%2Cwishlists%2CpublisherClass%2Cpublishers%2Cdevelopers'

# Параметры сбора данных
ROWS = 14000
ROWS_ON_PAGE = 50
COUNT_PAGES = int(ROWS / ROWS_ON_PAGE)

# Настраиваем сохранение csv-файлов в директории и установку самого файла в формате csv
UPLOADED_FILE = Path('gamalytic_games.csv')
DIR_DOWNLOADED = Path.cwd() / 'gamalytic_dl'

# Кнопка перехода на страницу и переменная для нахождения первой игры на странице (дял отслеживания переключения и контроля страниц)
BUTTON_NXT = 'button[aria-label="Go to next page"]'
FIRST_GAMENAME_ON_PAGE = 'div[role="row"][data-rowindex] [data-field="name"]'

def start_driver(dir_downloaded):
    dir_downloaded.mkdir(parents=True, exist_ok=True)
    options = Options()
    options.add_argument('--headless=new') # Запуск Chrome на фоне (окно не видно) дял удобства

    browser_settings = {
        'download.default_directory': str(dir_downloaded), # Указываем папку, куда сохранять скачанные файлы
        'download.prompt_for_download': False, # Отключаем всплывающее окно 'Выберите, куда сохранить файл'
        'download.directory_upgrade': True, # Используем для разрешения Chrome изменять папку загрузки на новую
        'safebrowsing.enabled': True, # Включаем безопасный режим, чтобы Chrome не блокировал загрузку файлов
    }
    options.add_experimental_option('prefs', browser_settings)

    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

# Функция ищет файл с максимальным временем изменения
def find_newest_csv_file(dir_downloaded):
    csv_files = list(dir_downloaded.glob('*.csv'))
    if len(csv_files) == 0: 
        return None
    return max(csv_files, key=lambda file: file.stat().st_mtime)

def click_to_download(driver, wait):
    button = find_menu(driver, wait)
    driver.execute_script('arguments[0].click()', button)
    time.sleep(0.3)
    web_elem = wait.until(ec.element_to_be_clickable((By.XPATH, '//li[text()="Download as CSV" or .//span[normalize-space()="Download as CSV"] ]')))
    t = time.time() # Запись времени 
    driver.execute_script('arguments[0].click()', web_elem)
    return t

def wait_for_dl(dir_downloaded, before):
    t0 = time.time()
    while time.time() - t0 < 60:
        if any(dir_downloaded.glob("*.crdownload")):
            time.sleep(0.3)
            continue
        f = find_newest_csv_file(dir_downloaded)
        if f and f.stat().st_mtime > before:
            return f
        time.sleep(0.2)

# Добавляет временно скаченный csv в итоговый
def append_csv(part_csv, out_csv, header):
    input_file = part_csv.open('r', encoding='utf-8', newline='')
    output_file = out_csv.open('a', encoding='utf-8', newline='')
    reader = csv.reader(input_file)
    writer = csv.writer(output_file)
    name_input_columns = next(reader, None)
    if not header and name_input_columns:
        writer.writerow(name_input_columns)
        header = True
    for line in reader:
        writer.writerow(line)
    input_file.close()
    output_file.close()
    return header

# Ищет кнопку меню (3 точки) для установки файла
def find_menu(driver, wait):
    el = wait.until(ec.element_to_be_clickable((By.XPATH, '//button[.//*[contains(@data-testid,"MoreVert")]]')))
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    return el

DIR_DOWNLOADED.mkdir(parents=True, exist_ok=True) # создаём папку
for path in DIR_DOWNLOADED.glob('*'):
    path.unlink(missing_ok=True) # удаляем временные csv
UPLOADED_FILE.write_text('', encoding='utf-8')


driver = start_driver(DIR_DOWNLOADED)
wait = WebDriverWait(driver, 15)
driver.get(URL)
wait.until(ec.presence_of_element_located((By.CSS_SELECTOR, '.MuiDataGrid-virtualScrollerRenderZone')))

header = False
for page in range(COUNT_PAGES):
    first_name = wait.until(ec.presence_of_element_located((By.CSS_SELECTOR, FIRST_GAMENAME_ON_PAGE))).text
    ts = click_to_download(driver, wait)
    csv_part = wait_for_dl(DIR_DOWNLOADED, ts)
    header = append_csv(csv_part, UPLOADED_FILE, header) # Проверяет, есть ли названия столбцов, добавляет временный csv в итоговый
    csv_part.unlink(missing_ok=True)
    next_btn = wait.until(ec.element_to_be_clickable((By.CSS_SELECTOR, BUTTON_NXT)))
    next_btn.click()
    wait.until_not(ec.text_to_be_present_in_element((By.CSS_SELECTOR, FIRST_GAMENAME_ON_PAGE), first_name))


driver.quit()
for path in DIR_DOWNLOADED.glob("*"): 
    path.unlink(missing_ok=True)